In [82]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string

In [2]:
comments_df = pd.read_csv('../data/comments.csv')
replies_df = pd.read_csv('../data/replies.csv')

In [3]:
all_comments_df = pd.concat([comments_df, replies_df], axis=0)

In [4]:
all_comments_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 766 entries, 0 to 588
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   comment         766 non-null    object 
 1   comment_id      766 non-null    object 
 2   parent_comment  589 non-null    object 
 3   user            766 non-null    object 
 4   user_avatar     766 non-null    object 
 5   user_page       766 non-null    object 
 6   comment_time    766 non-null    object 
 7   likes           766 non-null    int64  
 8   replies         177 non-null    float64
dtypes: float64(1), int64(1), object(7)
memory usage: 59.8+ KB


In [5]:
# replace the NaN's in the reply column with 'n/a' for comments that are comment threads
all_comments_df['parent_comment'] = all_comments_df['parent_comment'].fillna(value='n/a')

In [6]:
# replace the NaN's in the replies column with a value of 0 since these are replies to comments
all_comments_df['replies'] = all_comments_df['replies'].fillna(value=0)

In [7]:
# convert the comment_time column into a datetime format
all_comments_df['comment_time'] = pd.to_datetime(all_comments_df['comment_time'])

In [8]:
# convert certain columns to save space and perform quicker
all_comments_df = all_comments_df.astype({'likes': 'int32', 'replies': 'int32'})

In [20]:
singh_comments = all_comments_df[all_comments_df['user'] == '@asingha94']

In [76]:
words  = []

# apply a lambda (temproray) function to split each comment into a list of words where the split is done on spaces and punctuation marks, then use a for loop to append all the words into a list of comments, one comment at a time
#for comment in singh_comments['comment'].apply(lambda x: re.split(r'\W+', x)):
#    words += comment

# apply a lambda (temproray) function to split each comment into a list of words, then use a for loop to append all the words into a list of comments, one comment at a time
for comment in singh_comments['comment'].apply(lambda x: x.split()):
    words += comment

In [94]:
# create a mapping table that won't replace any characters but will remove punctuation characters
translate_table = str.maketrans('', '', string.punctuation)

# apply the mapping table to the list of words to strip all the punctuation
cleaned_words = [word.translate(translate_table) for word in words]

In [98]:
# make everything lower case
cleaned_words = [word.lower() for word in cleaned_words]